<a href="https://colab.research.google.com/github/yeesem/Advanced-Tensorflow-Specialization/blob/main/Fully_CNN_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import os
import zipfile
import PIL.Image,PIL.ImageFont,PIL.ImageDraw
import numpy as np

import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import seaborn as sns

# Download the Dataset

In [2]:
# download the dataset (zipped file)
!wget https://storage.googleapis.com/learning-datasets/fcnn-dataset.zip  -O /tmp/fcnn-dataset.zip

--2024-06-11 10:51:19--  https://storage.googleapis.com/learning-datasets/fcnn-dataset.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.207, 142.250.157.207, 142.251.8.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125577577 (120M) [application/zip]
Saving to: ‘/tmp/fcnn-dataset.zip’

/tmp/fcnn-dataset.z 100%[===================>] 119.76M  16.1MB/s    in 8.5s    

2024-06-11 10:51:27 (14.1 MB/s) - ‘/tmp/fcnn-dataset.zip’ saved [125577577/125577577]



In [3]:
local_zip = '/tmp/fcnn-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [4]:
# pixel labels in the video frames
class_names = ['sky', 'building','column/pole', 'road', 'side walk', 'vegetation', 'traffic light', 'fence', 'vehicle', 'pedestrian', 'byciclist', 'void']

# Load and Prepare the Dataset

In [5]:
def map_filename_to_image_and_mask(t_filename,a_filename,height = 224,width = 224):
  # Convert image and mask files to tensor
  img_raw = tf.io.read_file(t_filename)
  anno_raw = tf.io.read_file(a_filename)
  image = tf.image.decode_png(img_raw)
  annotation = tf.image.decode_png(anno_raw)

  # Resize image and segmentation mask
  image = tf.image.resize(image,(height,width,))
  annotation = tf.image.resize(annotation,(height,width,))
  image = tf.reshape(image,(height,width,3,))
  annotation = tf.cast(annotation,dtype = tf.int32)
  annotation = tf.reshape(annotation,(height,width,1,))
  stack_list = []

  # Reshape segmentation masks
  for c in range(len(class_names)):
    # mask = tf.equal(annotation[:,:,0], tf.constant(c)) is creating a binary mask where each element
    # in the mask is True if the corresponding element in annotation[:,:,0] is equal to the class c,
    # and False otherwise.
    # tf.equal(annotation[:,:,0], tf.constant(c)): This function compares each element in the 2D array
    # annotation[:,:,0] with the constant c. It returns a new 2D tensor of the same shape where each element
    # is True if the corresponding element in annotation[:,:,0] is equal to c, and False otherwise.
    mask = tf.equal(annotation[:,:,0],tf.constant(c))
    stack_list.append(tf.cast(mask,dtype = tf.int32))

  annotation = tf.stack(stack_list,axis = 2)

  # Normalize pixels in the input image
  image = image / 127.5
  # shift pixel value
  # This line shifts the scaled pixel values to the range [-1, 1].
  image -= 1

  return image,annotation

In [6]:
# show folders inside the dataset you downloaded
!ls /tmp/fcnn/dataset1

ls: cannot access '/tmp/fcnn/dataset1': No such file or directory
